In [1]:
from pyspark.sql import SparkSession


In [2]:
spk = SparkSession.builder.master('local[1]').appName('read-csv').getOrCreate()

22/05/20 18:55:44 WARN Utils: Your hostname, faridatl-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/20 18:55:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 18:55:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
arrests = spk.read.csv(path='/home/faridatl/Downloads/NYPD_ArrestsDataofficial.csv', sep=',', 
                  header=True, quote='"', inferSchema=True)

In [4]:
arrests.show(5)
arrests.printSchema() ## describes what kind of data is in dataframe 

+----------+-----------+-----+------------+-----+----------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+----------------+-------------+--------------------+---------+-------------------+------------------+----------------------+----------------+
|ARREST_KEY|ARREST_DATE|PD_CD|     PD_DESC|KY_CD| OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|        Latitude|    Longitude|             Lon_Lat|Zip Codes|Community Districts|Borough Boundaries|City Council Districts|Police Precincts|
+----------+-----------+-----+------------+-----+----------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+----------------+-------------+--------------------+---------+-------------------+------------------+----------------------+----------------+
| 192799737|  1/26/201

In [ ]:
'OFNS_DESC' 'PERP_RACE' 'PERP_SEX' 'ARREST_BORO' 'LAW_CAT_CD'

## Create offenses dataframe then save file to local

In [6]:
offenses = arrests.select('OFNS_DESC')
offenses.count()

747615

In [7]:
offenses = offenses.na.drop()
offenses.count()

745724

In [8]:
#rdd = spk.sparkContext.parallelize(offenses)
#type(offenses)
offenses.write.csv('offenses1.txt')

## Explore string elements with groupBy function

In [10]:
arrests.groupBy('OFNs_DESC').count().sort('count', ascending=False).show()

+--------------------+------+
|           OFNs_DESC| count|
+--------------------+------+
|     DANGEROUS DRUGS|102290|
|ASSAULT 3 & RELAT...|100202|
|       PETIT LARCENY| 68065|
|VEHICLE AND TRAFF...| 54178|
|      FELONY ASSAULT| 45410|
|MISCELLANEOUS PEN...| 37197|
|       GRAND LARCENY| 30120|
|   DANGEROUS WEAPONS| 28300|
|             ROBBERY| 27864|
|OTHER OFFENSES RE...| 27270|
|CRIMINAL MISCHIEF...| 20819|
|INTOXICATED & IMP...| 16196|
|OFFENSES AGAINST ...| 16098|
|             FORGERY| 15661|
|   CRIMINAL TRESPASS| 13440|
|            BURGLARY| 13315|
|OTHER TRAFFIC INF...| 11408|
|CRIMINAL MISCHIEF...| 10572|
|          SEX CRIMES| 10335|
|OFF. AGNST PUB OR...|  8015|
+--------------------+------+
only showing top 20 rows



In [8]:
arrests.groupBy('ARREST_BORO','OFNs_DESC').count().sort('count', ascending=False).show()

+-----------+--------------------+-----+
|ARREST_BORO|           OFNs_DESC|count|
+-----------+--------------------+-----+
|          K|     DANGEROUS DRUGS|29109|
|          B|ASSAULT 3 & RELAT...|28580|
|          B|     DANGEROUS DRUGS|27313|
|          M|       PETIT LARCENY|26773|
|          K|ASSAULT 3 & RELAT...|26207|
|          M|     DANGEROUS DRUGS|25785|
|          Q|ASSAULT 3 & RELAT...|22118|
|          M|ASSAULT 3 & RELAT...|19382|
|          Q|     DANGEROUS DRUGS|15299|
|          K|       PETIT LARCENY|15287|
|          K|MISCELLANEOUS PEN...|14051|
|          K|      FELONY ASSAULT|13579|
|          M|VEHICLE AND TRAFF...|13465|
|          M|       GRAND LARCENY|13224|
|          Q|VEHICLE AND TRAFF...|12945|
|          K|VEHICLE AND TRAFF...|12691|
|          B|VEHICLE AND TRAFF...|12412|
|          B|      FELONY ASSAULT|12064|
|          B|       PETIT LARCENY|11813|
|          M|OTHER OFFENSES RE...|11313|
+-----------+--------------------+-----+
only showing top

In [9]:
arrests.groupBy('ARREST_BORO', 'PERP_RACE','PERP_SEX','OFNS_DESC').count().sort('count', ascending=False).show() # BROOKLYN has the most

+-----------+--------------+--------+--------------------+-----+
|ARREST_BORO|     PERP_RACE|PERP_SEX|           OFNS_DESC|count|
+-----------+--------------+--------+--------------------+-----+
|          K|         BLACK|       M|     DANGEROUS DRUGS|13455|
|          K|         BLACK|       M|ASSAULT 3 & RELAT...|11137|
|          M|         BLACK|       M|     DANGEROUS DRUGS|10658|
|          B|         BLACK|       M|ASSAULT 3 & RELAT...| 9445|
|          B|         BLACK|       M|     DANGEROUS DRUGS| 9237|
|          M|         BLACK|       M|       PETIT LARCENY| 9090|
|          B|WHITE HISPANIC|       M|     DANGEROUS DRUGS| 8802|
|          K|         BLACK|       M|MISCELLANEOUS PEN...| 7117|
|          B|WHITE HISPANIC|       M|ASSAULT 3 & RELAT...| 6568|
|          M|         BLACK|       M|ASSAULT 3 & RELAT...| 6291|
|          K|         BLACK|       M|VEHICLE AND TRAFF...| 5960|
|          M|WHITE HISPANIC|       M|     DANGEROUS DRUGS| 5942|
|          K|         BLA

In [11]:
arrests.groupBy('PERP_RACE').count().sort('count', ascending=False).show()

+--------------------+------+
|           PERP_RACE| count|
+--------------------+------+
|               BLACK|355864|
|      WHITE HISPANIC|188813|
|               WHITE| 90513|
|      BLACK HISPANIC| 64714|
|ASIAN / PACIFIC I...| 39975|
|             UNKNOWN|  5700|
|AMERICAN INDIAN/A...|  2036|
+--------------------+------+



In [19]:
arrests.groupBy('PERP_SEX').count().sort('count', ascending=False).show()

+--------+------+
|PERP_SEX| count|
+--------+------+
|       M|613586|
|       F|134029|
+--------+------+



In [55]:
arrests.groupBy('AGE_GROUP').count().sort('count', ascending=False).show()

+---------+------+
|AGE_GROUP| count|
+---------+------+
|    25-44|386858|
|    18-24|165878|
|    45-64|147530|
|      <18| 38381|
|      65+|  8968|
+---------+------+

